# Extraction of local variables



In [ ]:
import locals_variables

locals_variables.main(first_year_train=1950,     #Filenames of train and test files should be set in local_variables.py according to the downloaded data
            last_year_train=2010,
            first_year_test=2011,
         last_year=2022,
         local_latitude=45.93,
         local_longitude=9.22,
         first_clima=1980,
         last_clima=2010,
         path_predictors='./Data/Resolution_0_5/',
         path_output='./Data/Test/')


: 